In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
vessel_name = 'OOCL_NEW_YORK'

# 전항지 출항 확인

전항지 출항 지점 확인

In [3]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [4]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

# 시각화 툴로 processed된 것들 확인

항로 클러스터링

In [5]:
# dense_upper_china_list = ['COSCO_FOS_VRKR6_23processed'
# ,'AKA_BHUM_9V7282_7processed'
# , 'COSCO_FOS_VRKR6_25processed'
# , 'MSC_LA_SPEZIA_DIGX2_2processed'
# , 'OOCL_NEW_YORK_VRJS2_2processed'
# , 'MSC_WASHINGTON_5LDP4_4processed'
# , 'MSC_SOFIA_PAZ_D5FL9_2processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_26processed'
# , 'OOCL_HAMBURG_VRZK9_6processed'
# , 'SEAMAX_STRATFORD_V7EP2_6processed'
# , 'OOCL_LUXEMBOURG_VRGO3_5processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_25processed'
# , 'MSC_LUCIANA_3FTY3_4processed'
# , 'MSC_RENEE_VRPL2_5processed'
# , 'COSCO_FOS_VRKR6_24processed'
# , 'COSCO_FOS_VRKR6_20processed'
# , 'COSCO_FOS_VRKR6_26processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_23processed'
# , 'OOCL_HAMBURG_VRZK9_5processed']

# OOCL_NEW_YORK_voyages_list = [
#     'COSCO_FOS_VRKR6_2022_20processed', 'COSCO_FOS_VRKR6_2022_23processed',
#     'COSCO_FOS_VRKR6_2022_24processed', 'COSCO_FOS_VRKR6_2022_25processed',
#     'AKA_BHUM_9V7282_2022_7processed', 'COSCO_FOS_VRKR6_2022_27processed',
#     'COSCO_FOS_VRKR6_2022_28processed', 'AKA_BHUM_9V7282_2022_7processed',
#     'COSCO_FOS_VRKR6_2022_30processed'
# ]

OOCL_NEW_YORK_voyages_list = [
    'AKA_BHUM_9V7282_2022_7processed', 'AKA_BHUM_9V7282_2022_8processed'
]
OOCL_NEW_YORK_voyages_list = [
    'OOCL_NEW_YORK_VRJS2_2022_1processed',
    'OOCL_NEW_YORK_VRJS2_2022_2processed',
    'OOCL_NEW_YORK_VRJS2_2022_3processed'
]

OOCL_LUXEMBOURG_voyages_list = [
    'OOCL_LUXEMBOURG_VRGO3_2022_5processed',
    'OOCL_LUXEMBOURG_VRGO3_2022_6processed'
]

OOCL_GENOA_voyages_list = [
    'OOCL_GENOA_VROK9_2022_3processed', 'OOCL_GENOA_VROK9_2022_5processed'
]

OOCL_HAMBURG_voyages_list = [
    'OOCL_HAMBURG_VRZK9_2022_5processed', 'OOCL_HAMBURG_VRZK9_2022_6processed'
]

SEAMAX_STRATFORD_voyages_list = [
    'SEAMAX_STRATFORD_V7EP2_2022_6processed',
    'SEAMAX_STRATFORD_V7EP2_2022_7processed'
]

ZIM_CHARLESTORN_voyages_list = [
    'ZIM_CHARLESTON_VRUC3_2022_6processed',
    'ZIM_CHARLESTON_VRUC3_2022_7processed'
]

single_voyage_list = [
    'MSC_ARIANE_3FDA3_2022_1processed', 'MSC_DANIT_3FZU8_2022_2processed',
    'MSC_EVA_3EVM7_2022_1processed', 'MSC_EXPRESS_III_A8LG7_2022_1processed',
    'MSC_KANOKO_D5TM3_2023_1processed', 'MSC_LA_SPEZIA_DIGX2_2022_2processed',
    'MSC_VANESSA_HOWD_2023_1processed', 'MSC_VIRGO_CQEB9_2023_2processed',
    'MSC_SOFIA_PAZ_D5FL9_2022_2processed',
    'MSC_WASHINGTON_5LDP4_2022_4processed',
    'MSC_RAVENNA_A8ZU8_2022_3processed', 'MSC_VANDYA_3FDW4_2022_2processed',
    'MSC_RENEE_VRPL2_2022_5processed', 'SEASPAN_OCEANIA_VRBI2_2023_1processed',
    'SEASPAN_BREEZE_VRNL8_2023_1processed', 'VALENCE_9HA3398_2023_1processed'
]

In [6]:
upper_china_path_dir = 'voy_path/upper_china/processed/'

In [7]:
# 가는 길인데 굳이 오래 돌아갈 필요는 없잖아 알고리즘
def optimal_next_point(current_position, shortest_point_idx, shortest_distance,
                       path_df):
    posi_c = current_position
    latitude_i = path_df.loc[shortest_point_idx, "latitude"]
    longitude_i = path_df.loc[shortest_point_idx, "longitude"]
    posi_i = (latitude_i, longitude_i)
    min_d = shortest_distance
    n = shortest_point_idx + 1
    while True:
        try:
            latitude_n = path_df.loc[n, "latitude"]
            longitude_n = path_df.loc[n, "longitude"]
            posi_n = (latitude_n, longitude_n)
            distance = haversine(posi_c, posi_n, unit="km")
            if n == shortest_point_idx + 1:
                path_distance = haversine(posi_i, posi_n, unit="km")
                paths_distance = path_distance
            else:
                path_distance = haversine(posi_n, posi_previous_n, unit="km")
                paths_distance += path_distance

            if distance < (paths_distance + min_d):
                optimal_next_point_idx = n
                return optimal_next_point_idx
            elif n == path_df.index[-1]:
                print("no better optimal_point")
                return shortest_point_idx + 1
            else:
                n = n + 1
                posi_previous_n = posi_n
        except:
            print("no better optimal_point")
            return shortest_point_idx + 1

In [8]:
# 인덱스 번호를 가지고 항로의 segment 번호를 찾음
# 만약 segment의 마지막 번호라면 seg_no를 하나 더한다.

def find_segment_no(optimal_point_idx, path_df, the_number_of_segment):
    seg_no = optimal_point_idx // (len(path_df) / the_number_of_segment)
    next_no = (optimal_point_idx + 1) // (len(path_df) / the_number_of_segment)
    if seg_no != next_no:
        return int(next_no)
    else:
        return int(seg_no)

In [9]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [10]:
def cal_duration(speed, distance):
    hour = int(distance // km_h)
    minute = int(
        (distance / km_h - hour) * 60)
    second = ((distance / km_h - hour) * 60 - minute) * 60
    duration = datetime.timedelta(hours=hour,minutes=minute, seconds=second)
    return duration

# Tianjin to Busan PNIT

## OOCL_NEW_YORK

In [11]:
coordinates_array = pd.DataFrame(data={"latitude" : [],"longitude" : []})

for i in OOCL_NEW_YORK_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    coordinates_array = coordinates_array.append(df[["latitude","longitude"]], ignore_index=True)

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from shapely.geometry import LineString

In [14]:
from scipy import interpolate
import pandas as pd

## below 3 knot 제거

In [15]:
interpolation_number = "2000"

### interpolation 2000

In [16]:
max_speed_list = []
min_speed_list = []
for i in OOCL_NEW_YORK_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in OOCL_NEW_YORK_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.433653846153845
보간법 전 일반 속도 표준편차: 1.577918659655788
보간법 전 저속 제거 평균 속도 : 9.433653846153845
보간법 전 저속 제거 속도 표준편차: 1.577918659655788
-
보간법 후 일반 평균 속도 : 9.055060851234295
보간법 후 일반 속도 표준편차: 1.5068284261246045
보간법 후 저속 제거 평균 속도 : 9.055060851234295
보간법 후 저속 제거 속도 표준편차: 1.5068284261246045
보간법 전 일반 평균 속도 : 13.148247978436657
보간법 전 일반 속도 표준편차: 1.421016686831533
보간법 전 저속 제거 평균 속도 : 13.148247978436657
보간법 전 저속 제거 속도 표준편차: 1.421016686831533
-
보간법 후 일반 평균 속도 : 13.336259410642818
보간법 후 일반 속도 표준편차: 1.304434315939391
보간법 후 저속 제거 평균 속도 : 13.336259410642818
보간법 후 저속 제거 속도 표준편차: 1.304434315939391
보간법 전 일반 평균 속도 : 12.789285714285715
보간법 전 일반 속도 표준편차: 2.479073208239914
보간법 전 저속 제거 평균 속도 : 12.789285714285715
보간법 전 저속 제거 속도 표준편차: 2.479073208239914
-
보간법 후 일반 평균 속도 : 13.859110928864379
보간법 후 일반 속도 표준편차: 2.9169778389505647
보간법 후 저속 제거 평균 속도 : 13.859110928864379
보간법 후 저속 제거 속도 표준편차: 2.9169778389505647


In [17]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['OOCL_NEW_YORK_VRJS2_2022_1',
 'OOCL_NEW_YORK_VRJS2_2022_2',
 'OOCL_NEW_YORK_VRJS2_2022_3']

In [18]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in OOCL_NEW_YORK_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
    elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        high_speed_path_data.append(df)

In [19]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [20]:
result_df_dict=  {
    "experiment_no" : [],
    "mean_speed" : [],
    "max_speed" : []
}

# ETA 실험 (interpolation 2000), OOCL_NEW_YORK 1항차 저속

In [21]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.667818e+09,117.963300,38.925933,11.000000
1,1.667818e+09,117.977437,38.923934,11.084237
2,1.667819e+09,117.991574,38.921934,11.168474
3,1.667819e+09,118.005711,38.919934,11.252711
4,1.667819e+09,118.019430,38.917983,11.188368
...,...,...,...,...
1995,1.668138e+09,128.866992,34.894323,8.503004
1996,1.668138e+09,128.865347,34.901075,8.261692
1997,1.668138e+09,128.862609,34.908995,7.941128
1998,1.668138e+09,128.859871,34.916914,7.620564


In [22]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_1processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 90

In [23]:
the_number_of_segment = 90

In [24]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [25]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [26]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.144098723474787
10.583898287998867
12.781331943079305
13.316882587546466
13.453192059604982
13.45553703321939
13.12938370243301
12.772073584506359
12.400855535412193
12.45691298036514
12.894604103403731
13.026769782257894
13.144667110709879
13.000763779442826
13.034449945673925
13.240303723439741
13.533528584204612
13.564501506126176
13.467830757733037
13.376689348478715
13.299521148483874
13.204969089680587
13.054967583915175
13.017715374151761
12.991469430551511
13.064976018931231
13.13232641761145
13.195751875268098
13.259177332927129
13.322602790586132
13.382092960363963
13.418683142147545
13.488685380158474
13.485954521267116
13.48322366237481
13.417286402747362
13.264104969293129
13.144118041797082
13.10593298068316
13.156733370229984
13.26080146248372
13.248552344584455
13.3258628601949
13.428203990499691
13.5457109948201
13.411184755612153
13.299721685266201
13.07621622363062
12.89503390049416
12.973162654016251
12.564438871272971
12.147485447202898
12.071138703276098
12.020

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [27]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_1processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [28]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_1".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [29]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 4 days 09:58:03.416934
평균: 2 days 02:49:52.021219578


In [30]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
115,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200900,127.8664,33.9982,10.2,43.0,2022-09-20 21:21:53.231356,2022-09-20 12:08:46,2022-09-20 11:43:50,under way using engine,138.914731,228.469157,2022-09-21 03:30:00.994418,2022-09-21 04:22:27.234942,0 days 00:52:26.240524
120,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,128.3194,34.3035,11.0,56.0,2022-09-20 23:52:04.976523,2022-09-20 14:51:16,2022-09-20 02:19:52,under way using engine,87.056398,144.428112,2022-09-21 03:17:58.735119,2022-09-21 04:22:27.234942,0 days 01:04:28.499823
113,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200900,127.7637,33.8957,10.2,46.0,2022-09-20 20:21:47.421845,2022-09-20 11:21:35,2022-09-19 03:19:52,under way using engine,153.732875,252.840142,2022-09-21 03:15:17.135979,2022-09-21 04:22:27.234942,0 days 01:07:10.098963
117,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,128.0478,34.1579,11.1,40.0,2022-09-20 22:21:57.978812,2022-09-20 13:21:26,2022-09-20 01:07:50,under way using engine,114.624580,190.517057,2022-09-21 03:05:46.086875,2022-09-21 04:22:27.234942,0 days 01:16:41.148067
116,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,127.9739,34.0858,11.4,44.0,2022-09-20 21:51:55.060383,2022-09-20 12:50:47,2022-09-20 12:37:50,under way using engine,125.129767,209.333676,2022-09-21 03:03:10.834976,2022-09-21 04:22:27.234942,0 days 01:19:16.399966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9191201,120.7079,38.5614,8.4,89.0,2022-09-15 01:36:47.012779,2022-09-14 16:24:18,2022-09-14 04:25:51,under way using engine,825.124440,1396.502679,2022-09-16 21:29:59.977418,2022-09-21 04:22:27.234942,4 days 06:52:27.257524
4,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9191201,120.2620,38.5983,9.3,90.0,2022-09-15 00:06:39.552593,2022-09-14 13:59:06,2022-09-14 02:07:53,under way using engine,861.791634,1424.531580,2022-09-16 21:29:16.776502,2022-09-21 04:22:27.234942,4 days 06:53:10.458440
1,OOCL NEW YORK,477274700,9198109.0,5770,HKG CN ZIN,9121000,118.5881,38.8012,7.7,94.0,2022-09-14 13:46:22.415649,2022-09-14 04:24:46,2022-09-13 10:25:51,under way using engine,1003.951371,1756.661582,2022-09-16 18:42:08.690074,2022-09-21 04:22:27.234942,4 days 09:40:18.544868
0,OOCL NEW YORK,477274700,9198109.0,5770,HKG CN ZIN,9121000,118.2459,38.8466,9.7,97.0,2022-09-14 12:46:11.783705,2022-09-14 02:33:56,2022-09-13 10:25:51,under way using engine,1033.365335,1700.726451,2022-09-16 18:27:51.145111,2022-09-21 04:22:27.234942,4 days 09:54:36.089831


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [31]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_1processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [32]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_1".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [33]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 4 days 09:58:03.416934
평균: 1 days 21:30:16.913890


In [34]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
115,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200900,127.8664,33.9982,10.2,43.0,2022-09-20 21:21:53.231356,2022-09-20 12:08:46,2022-09-20 11:43:50,under way using engine,138.914731,228.469157,2022-09-21 03:30:00.994418,2022-09-21 04:22:27.234942,0 days 00:52:26.240524
120,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,128.3194,34.3035,11.0,56.0,2022-09-20 23:52:04.976523,2022-09-20 14:51:16,2022-09-20 02:19:52,under way using engine,87.056398,144.428112,2022-09-21 03:17:58.735119,2022-09-21 04:22:27.234942,0 days 01:04:28.499823
113,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200900,127.7637,33.8957,10.2,46.0,2022-09-20 20:21:47.421845,2022-09-20 11:21:35,2022-09-19 03:19:52,under way using engine,153.732875,252.840142,2022-09-21 03:15:17.135979,2022-09-21 04:22:27.234942,0 days 01:07:10.098963
117,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,128.0478,34.1579,11.1,40.0,2022-09-20 22:21:57.978812,2022-09-20 13:21:26,2022-09-20 01:07:50,under way using engine,114.624580,190.517057,2022-09-21 03:05:46.086875,2022-09-21 04:22:27.234942,0 days 01:16:41.148067
116,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9200800,127.9739,34.0858,11.4,44.0,2022-09-20 21:51:55.060383,2022-09-20 12:50:47,2022-09-20 12:37:50,under way using engine,125.129767,209.333676,2022-09-21 03:03:10.834976,2022-09-21 04:22:27.234942,0 days 01:19:16.399966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,9191201,120.2620,38.5983,9.3,90.0,2022-09-15 00:06:39.552593,2022-09-14 13:59:06,2022-09-14 02:07:53,under way using engine,861.791634,1424.531580,2022-09-16 21:29:16.776502,2022-09-21 04:22:27.234942,4 days 06:53:10.458440
3,OOCL NEW YORK,477274700,9198109.0,5770,HKG CN ZIN,9121000,118.7645,38.7795,6.9,92.0,2022-09-14 14:53:58.476930,2022-09-14 05:34:24,2022-09-13 10:25:51,under way using engine,988.878505,1830.689186,2022-09-16 19:01:55.961882,2022-09-21 04:22:27.234942,4 days 09:20:31.273060
1,OOCL NEW YORK,477274700,9198109.0,5770,HKG CN ZIN,9121000,118.5881,38.8012,7.7,94.0,2022-09-14 13:46:22.415649,2022-09-14 04:24:46,2022-09-13 10:25:51,under way using engine,1003.951371,1756.661582,2022-09-16 18:42:08.690074,2022-09-21 04:22:27.234942,4 days 09:40:18.544868
0,OOCL NEW YORK,477274700,9198109.0,5770,HKG CN ZIN,9121000,118.2459,38.8466,9.7,97.0,2022-09-14 12:46:11.783705,2022-09-14 02:33:56,2022-09-13 10:25:51,under way using engine,1033.365335,1700.726451,2022-09-16 18:27:51.145111,2022-09-21 04:22:27.234942,4 days 09:54:36.089831


# ETA 실험 (interpolation 2000), OOCL_NEW_YORK 2항차 저속

In [35]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.667818e+09,117.963300,38.925933,11.000000
1,1.667818e+09,117.977437,38.923934,11.084237
2,1.667819e+09,117.991574,38.921934,11.168474
3,1.667819e+09,118.005711,38.919934,11.252711
4,1.667819e+09,118.019430,38.917983,11.188368
...,...,...,...,...
1995,1.668138e+09,128.866992,34.894323,8.503004
1996,1.668138e+09,128.865347,34.901075,8.261692
1997,1.668138e+09,128.862609,34.908995,7.941128
1998,1.668138e+09,128.859871,34.916914,7.620564


In [36]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_2processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 90

In [37]:
the_number_of_segment = 90

In [38]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [39]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [40]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.144098723474787
10.583898287998867
12.781331943079305
13.316882587546466
13.453192059604982
13.45553703321939
13.12938370243301
12.772073584506359
12.400855535412193
12.45691298036514
12.894604103403731
13.026769782257894
13.144667110709879
13.000763779442826
13.034449945673925
13.240303723439741
13.533528584204612
13.564501506126176
13.467830757733037
13.376689348478715
13.299521148483874
13.204969089680587
13.054967583915175
13.017715374151761
12.991469430551511
13.064976018931231
13.13232641761145
13.195751875268098
13.259177332927129
13.322602790586132
13.382092960363963
13.418683142147545
13.488685380158474
13.485954521267116
13.48322366237481
13.417286402747362
13.264104969293129
13.144118041797082
13.10593298068316
13.156733370229984
13.26080146248372
13.248552344584455
13.3258628601949
13.428203990499691
13.5457109948201
13.411184755612153
13.299721685266201
13.07621622363062
12.89503390049416
12.973162654016251
12.564438871272971
12.147485447202898
12.071138703276098
12.020

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [41]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_2processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [42]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_2".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [43]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:46:59.237234
평균: 0 days 00:33:03.627164142


In [44]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
206,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,125.1033,34.3266,14.3,138.0,2022-11-10 17:08:22.363147,2022-11-10 08:07:24,2022-11-09 10:19:51,under way using engine,347.351934,654.647330,2022-11-11 12:28:55.612185,2022-11-11 12:29:16.869978,0 days 00:00:21.257793
149,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.3157,35.6904,13.8,128.0,2022-11-10 10:11:17.175921,2022-11-10 01:11:03,2022-11-09 10:19:51,under way using engine,416.312099,764.249642,2022-11-11 12:29:45.514227,2022-11-11 12:29:16.869978,0 days 00:00:28.644249
180,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.8548,34.7593,13.3,159.0,2022-11-10 14:57:09.387599,2022-11-10 05:56:24,2022-11-09 10:19:51,under way using engine,362.451081,649.390878,2022-11-11 12:29:53.115818,2022-11-11 12:29:16.869978,0 days 00:00:36.245840
90,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,122.9109,37.6049,14.4,111.0,2022-11-10 00:25:47.972716,2022-11-09 15:22:17,2022-11-09 11:07:53,under way using engine,605.069725,1146.621389,2022-11-11 12:30:16.858909,2022-11-11 12:29:16.869978,0 days 00:00:59.988931
186,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.9069,34.6568,13.0,163.0,2022-11-10 15:27:28.389722,2022-11-10 06:27:03,2022-11-09 10:19:51,under way using engine,358.854803,634.319739,2022-11-11 12:28:08.318553,2022-11-11 12:29:16.869978,0 days 00:01:08.551425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.2169,34.2692,11.8,65.0,2022-11-11 07:41:34.983984,2022-11-10 22:40:32,2022-11-10 10:25:50,under way using engine,95.422875,161.357194,2022-11-11 11:07:28.742580,2022-11-11 12:29:16.869978,0 days 01:21:48.127398
341,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.3725,34.3503,11.7,47.0,2022-11-11 08:32:03.964430,2022-11-10 23:27:10,2022-11-10 11:31:50,under way using engine,80.082431,135.025973,2022-11-11 11:03:37.069291,2022-11-11 12:29:16.869978,0 days 01:25:39.800687
330,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.2018,34.2624,11.8,65.0,2022-11-11 07:36:31.891423,2022-11-10 22:36:10,2022-11-10 10:25:50,under way using engine,96.842193,163.757219,2022-11-11 11:02:25.650019,2022-11-11 12:29:16.869978,0 days 01:26:51.219959
340,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.3637,34.3438,11.6,47.0,2022-11-11 08:27:00.960386,2022-11-10 23:24:10,2022-11-10 10:55:50,under way using engine,81.116266,136.393050,2022-11-11 10:58:34.065247,2022-11-11 12:29:16.869978,0 days 01:30:42.804731


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [45]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_2processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [46]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_2".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [47]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:46:59.237234
평균: 0 days 00:33:03.627164142


In [48]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
206,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,125.1033,34.3266,14.3,138.0,2022-11-10 17:08:22.363147,2022-11-10 08:07:24,2022-11-09 10:19:51,under way using engine,347.351934,654.647330,2022-11-11 12:28:55.612185,2022-11-11 12:29:16.869978,0 days 00:00:21.257793
149,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.3157,35.6904,13.8,128.0,2022-11-10 10:11:17.175921,2022-11-10 01:11:03,2022-11-09 10:19:51,under way using engine,416.312099,764.249642,2022-11-11 12:29:45.514227,2022-11-11 12:29:16.869978,0 days 00:00:28.644249
180,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.8548,34.7593,13.3,159.0,2022-11-10 14:57:09.387599,2022-11-10 05:56:24,2022-11-09 10:19:51,under way using engine,362.451081,649.390878,2022-11-11 12:29:53.115818,2022-11-11 12:29:16.869978,0 days 00:00:36.245840
90,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,122.9109,37.6049,14.4,111.0,2022-11-10 00:25:47.972716,2022-11-09 15:22:17,2022-11-09 11:07:53,under way using engine,605.069725,1146.621389,2022-11-11 12:30:16.858909,2022-11-11 12:29:16.869978,0 days 00:00:59.988931
186,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,124.9069,34.6568,13.0,163.0,2022-11-10 15:27:28.389722,2022-11-10 06:27:03,2022-11-09 10:19:51,under way using engine,358.854803,634.319739,2022-11-11 12:28:08.318553,2022-11-11 12:29:16.869978,0 days 00:01:08.551425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.2169,34.2692,11.8,65.0,2022-11-11 07:41:34.983984,2022-11-10 22:40:32,2022-11-10 10:25:50,under way using engine,95.422875,161.357194,2022-11-11 11:07:28.742580,2022-11-11 12:29:16.869978,0 days 01:21:48.127398
341,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.3725,34.3503,11.7,47.0,2022-11-11 08:32:03.964430,2022-11-10 23:27:10,2022-11-10 11:31:50,under way using engine,80.082431,135.025973,2022-11-11 11:03:37.069291,2022-11-11 12:29:16.869978,0 days 01:25:39.800687
330,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.2018,34.2624,11.8,65.0,2022-11-11 07:36:31.891423,2022-11-10 22:36:10,2022-11-10 10:25:50,under way using engine,96.842193,163.757219,2022-11-11 11:02:25.650019,2022-11-11 12:29:16.869978,0 days 01:26:51.219959
340,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,11110200,128.3637,34.3438,11.6,47.0,2022-11-11 08:27:00.960386,2022-11-10 23:24:10,2022-11-10 10:55:50,under way using engine,81.116266,136.393050,2022-11-11 10:58:34.065247,2022-11-11 12:29:16.869978,0 days 01:30:42.804731


# ETA 실험 (interpolation 2000), OOCL_NEW_YORK 3항차 저속

In [49]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_{vessel_name}_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.667818e+09,117.963300,38.925933,11.000000
1,1.667818e+09,117.977437,38.923934,11.084237
2,1.667819e+09,117.991574,38.921934,11.168474
3,1.667819e+09,118.005711,38.919934,11.252711
4,1.667819e+09,118.019430,38.917983,11.188368
...,...,...,...,...
1995,1.668138e+09,128.866992,34.894323,8.503004
1996,1.668138e+09,128.865347,34.901075,8.261692
1997,1.668138e+09,128.862609,34.908995,7.941128
1998,1.668138e+09,128.859871,34.916914,7.620564


## 저속 (7knot 미만 제거)

### segment 90

In [50]:
the_number_of_segment = 90

In [51]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [52]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [53]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.144098723474787
10.583898287998867
12.781331943079305
13.316882587546466
13.453192059604982
13.45553703321939
13.12938370243301
12.772073584506359
12.400855535412193
12.45691298036514
12.894604103403731
13.026769782257894
13.144667110709879
13.000763779442826
13.034449945673925
13.240303723439741
13.533528584204612
13.564501506126176
13.467830757733037
13.376689348478715
13.299521148483874
13.204969089680587
13.054967583915175
13.017715374151761
12.991469430551511
13.064976018931231
13.13232641761145
13.195751875268098
13.259177332927129
13.322602790586132
13.382092960363963
13.418683142147545
13.488685380158474
13.485954521267116
13.48322366237481
13.417286402747362
13.264104969293129
13.144118041797082
13.10593298068316
13.156733370229984
13.26080146248372
13.248552344584455
13.3258628601949
13.428203990499691
13.5457109948201
13.411184755612153
13.299721685266201
13.07621622363062
12.89503390049416
12.973162654016251
12.564438871272971
12.147485447202898
12.071138703276098
12.020

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [54]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_3processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [55]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}_voy_3".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [56]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:30:09.999751
평균: 0 days 02:10:09.274850572


In [57]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
29,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.6700,38.5495,16.3,98.0,2022-12-29 22:30:33.073635,2022-12-29 13:27:47,2022-12-29 01:13:54,under way using engine,827.490198,73.258111,2022-12-31 18:23:46.038274,2022-12-31 18:24:32.781137,0 days 00:00:46.742863
46,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.7492,38.1199,15.8,129.0,2022-12-30 02:06:06.993394,2022-12-29 17:02:23,2022-12-29 03:49:54,under way using engine,721.936194,61.917656,2022-12-31 18:23:29.222096,2022-12-31 18:24:32.781137,0 days 00:01:03.559041
36,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.0423,38.4777,16.8,116.0,2022-12-29 23:35:43.297860,2022-12-29 14:35:17,2022-12-29 02:25:52,under way using engine,794.954562,72.707756,2022-12-31 18:27:23.516833,2022-12-31 18:24:32.781137,0 days 00:02:50.735696
47,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.7724,38.1038,15.8,129.0,2022-12-30 02:11:06.990195,2022-12-29 17:08:03,2022-12-29 03:49:54,under way using engine,719.316671,61.692990,2022-12-31 18:28:29.218897,2022-12-31 18:24:32.781137,0 days 00:03:56.437760
30,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.7053,38.5453,16.2,90.0,2022-12-29 22:35:34.752117,2022-12-29 13:34:05,2022-12-29 01:13:54,under way using engine,824.531068,72.529152,2022-12-31 18:28:47.716756,2022-12-31 18:24:32.781137,0 days 00:04:14.935619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.1395,34.2045,11.1,38.0,2022-12-31 11:03:25.085435,2022-12-31 02:03:08,2022-12-31 12:55:50,under way using engine,105.413749,7.300324,2022-12-31 15:05:21.500620,2022-12-31 18:24:32.781137,0 days 03:19:11.280517
178,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.2265,34.2846,10.1,37.0,2022-12-31 11:38:30.616460,2022-12-31 02:37:48,2022-12-31 12:55:50,under way using engine,93.515179,6.406734,2022-12-31 15:04:24.375056,2022-12-31 18:24:32.781137,0 days 03:20:08.406081
170,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.1251,34.1926,11.0,39.0,2022-12-31 10:58:24.336759,2022-12-31 01:57:38,2022-12-31 12:55:50,under way using engine,107.263065,7.414639,2022-12-31 15:00:20.751944,2022-12-31 18:24:32.781137,0 days 03:24:12.029193
177,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.2152,34.2736,10.9,37.0,2022-12-31 11:33:29.691943,2022-12-31 02:32:47,2022-12-31 12:55:50,under way using engine,95.113479,6.563677,2022-12-31 14:59:23.450539,2022-12-31 18:24:32.781137,0 days 03:25:09.330598


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [58]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/OOCL_NEW_YORK_VRJS2_2022_3processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [59]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["timestamp"]
#     current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"timestamp"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}_voy_3".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [60]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:30:09.999751
평균: 0 days 02:09:49.049920592


In [61]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
29,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.6700,38.5495,16.3,98.0,2022-12-29 22:30:33.073635,2022-12-29 13:27:47,2022-12-29 01:13:54,under way using engine,827.490198,73.258111,2022-12-31 18:23:46.038274,2022-12-31 18:24:32.781137,0 days 00:00:46.742863
46,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.7492,38.1199,15.8,129.0,2022-12-30 02:06:06.993394,2022-12-29 17:02:23,2022-12-29 03:49:54,under way using engine,721.936194,61.917656,2022-12-31 18:23:29.222096,2022-12-31 18:24:32.781137,0 days 00:01:03.559041
36,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.0423,38.4777,16.8,116.0,2022-12-29 23:35:43.297860,2022-12-29 14:35:17,2022-12-29 02:25:52,under way using engine,794.954562,72.707756,2022-12-31 18:27:23.516833,2022-12-31 18:24:32.781137,0 days 00:02:50.735696
47,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,121.7724,38.1038,15.8,129.0,2022-12-30 02:11:06.990195,2022-12-29 17:08:03,2022-12-29 03:49:54,under way using engine,719.316671,61.692990,2022-12-31 18:28:29.218897,2022-12-31 18:24:32.781137,0 days 00:03:56.437760
30,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,120.7053,38.5453,16.2,90.0,2022-12-29 22:35:34.752117,2022-12-29 13:34:05,2022-12-29 01:13:54,under way using engine,824.531068,72.529152,2022-12-31 18:28:47.716756,2022-12-31 18:24:32.781137,0 days 00:04:14.935619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.1395,34.2045,11.1,38.0,2022-12-31 11:03:25.085435,2022-12-31 02:03:08,2022-12-31 12:55:50,under way using engine,105.413749,7.300324,2022-12-31 15:05:21.500620,2022-12-31 18:24:32.781137,0 days 03:19:11.280517
178,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.2265,34.2846,10.1,37.0,2022-12-31 11:38:30.616460,2022-12-31 02:37:48,2022-12-31 12:55:50,under way using engine,93.515179,6.406734,2022-12-31 15:04:24.375056,2022-12-31 18:24:32.781137,0 days 03:20:08.406081
170,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.1251,34.1926,11.0,39.0,2022-12-31 10:58:24.336759,2022-12-31 01:57:38,2022-12-31 12:55:50,under way using engine,107.263065,7.414639,2022-12-31 15:00:20.751944,2022-12-31 18:24:32.781137,0 days 03:24:12.029193
177,OOCL NEW YORK,477274700,9198109.0,5770,KRBNP,12310800,128.2152,34.2736,10.9,37.0,2022-12-31 11:33:29.691943,2022-12-31 02:32:47,2022-12-31 12:55:50,under way using engine,95.113479,6.563677,2022-12-31 14:59:23.450539,2022-12-31 18:24:32.781137,0 days 03:25:09.330598


# 결과 확인

In [62]:
result_pool = pd.DataFrame(result_df_dict)
result_pool

,experiment_no,mean_speed,max_speed
0,4_1_2_3_seg_90_voy_1,2 days 02:49:52.021219578,4 days 09:58:03.416934
1,4_2_1_3_seg_90_voy_1,1 days 21:30:16.913890,4 days 09:58:03.416934
2,4_1_2_3_seg_90_voy_2,0 days 00:33:03.627164142,0 days 02:46:59.237234
3,4_2_1_3_seg_90_voy_2,0 days 00:33:03.627164142,0 days 02:46:59.237234
4,4_1_2_3_seg_90_voy_3,0 days 02:10:09.274850572,0 days 03:30:09.999751
5,4_2_1_3_seg_90_voy_3,0 days 02:09:49.049920592,0 days 03:30:09.999751


In [63]:
result_pool.sort_values(by="mean_speed")

,experiment_no,mean_speed,max_speed
2,4_1_2_3_seg_90_voy_2,0 days 00:33:03.627164142,0 days 02:46:59.237234
3,4_2_1_3_seg_90_voy_2,0 days 00:33:03.627164142,0 days 02:46:59.237234
5,4_2_1_3_seg_90_voy_3,0 days 02:09:49.049920592,0 days 03:30:09.999751
4,4_1_2_3_seg_90_voy_3,0 days 02:10:09.274850572,0 days 03:30:09.999751
1,4_2_1_3_seg_90_voy_1,1 days 21:30:16.913890,4 days 09:58:03.416934
0,4_1_2_3_seg_90_voy_1,2 days 02:49:52.021219578,4 days 09:58:03.416934


In [64]:
for i in globals():
    if 'exp_result_df_' in i:
        globals()[i].to_csv(f"실험/230205_{vessel_name}_{i}", encoding="utf-8 sig", header=True, index=False)

In [65]:
# exp_result_df_4_1_2_3

In [66]:
# result_pool.to_csv(f"230126_result_pool_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)